# Data Cleaning of the Linkedin Data Science family Jobs Data 

In [1]:
# Importing Required Libraries
import pandas as pd
import numpy as np
import datetime 
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import re
from datetime import datetime, timedelta
from sklearn.impute import SimpleImputer

In [2]:
DS_jobs = pd.read_excel("Data_Scientist_Jobs.xlsx")
DS_jobs.head()

,Job_title,Company,Job_location,Post_time,Applicants_count,Job_description,Seniority_level,Employment_type,Job_function,Industries
0,Machine Learning Engineer,Patterned Learning AI,"San Francisco, CA",1 day ago,58,", Full-Time, Salary $60K-$70K\n\n\n\n\nMinimum...",Entry level,Full-time,Engineering and Information Technology,IT Services and IT Consulting
1,Machine Learning Engineer,Neural Magic,"Somerville, MA",1 month ago,131,Neural Magic is an early-stage AI software com...,Associate,Full-time,Engineering and Information Technology,"Technology, Information and Internet"
2,Data Scientist,Patterned Learning AI,"New York, United States",1 week ago,250,This is a remote position.\n\n\n\n\nEntry-Leve...,Entry level,Full-time,Engineering and Information Technology,IT Services and IT Consulting
3,Data Scientist,Patterned Learning AI,"San Francisco, CA",1 day ago,49,"Junior Data Scientist Engineer, Full-Time, Sal...",Entry level,Full-time,Engineering and Information Technology,IT Services and IT Consulting
4,Data Scientist,Patterned Learning AI,"New York, NY",17 hours ago,56,"Entry-Level Data Scientist Engineer - US, Full...",Entry level,Full-time,Engineering and Information Technology,IT Services and IT Consulting


In [3]:
DS_jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2346 entries, 0 to 2345
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Job_title         2346 non-null   object
 1   Company           2346 non-null   object
 2   Job_location      2346 non-null   object
 3   Post_time         2346 non-null   object
 4   Applicants_count  2346 non-null   int64 
 5   Job_description   2346 non-null   object
 6   Seniority_level   2115 non-null   object
 7   Employment_type   2332 non-null   object
 8   Job_function      2332 non-null   object
 9   Industries        2332 non-null   object
dtypes: int64(1), object(9)
memory usage: 183.4+ KB


**Missing values**

In [4]:
DS_jobs.isnull().sum()

Job_title             0
Company               0
Job_location          0
Post_time             0
Applicants_count      0
Job_description       0
Seniority_level     231
Employment_type      14
Job_function         14
Industries           14
dtype: int64

* We got missing values in the  Seniority_level, Employment_type, Job_function and Industries

* For now, I'm just using simple imputer and replacing the missing values with constant value as 'Missing'

In [5]:
# Filtering the columns with missing values
columns_with_missing = DS_jobs.columns[DS_jobs.isnull().any()]

# Using SimpleImputer to fill missing values with 'Missing'
imputer = SimpleImputer(strategy='constant', fill_value='Missing')
DS_jobs[columns_with_missing] = imputer.fit_transform(DS_jobs[columns_with_missing])

# Displaying the updated DataFrame
DS_jobs.isnull().sum()


Job_title           0
Company             0
Job_location        0
Post_time           0
Applicants_count    0
Job_description     0
Seniority_level     0
Employment_type     0
Job_function        0
Industries          0
dtype: int64

***

**Converting Job Posted Time to Date Time Format** 

* In the extracted data we got only days posted information which I have converted it into the datetime variable by subtracting it from the current date.  

In [4]:
DS_jobs['Post_time'].head(5)

0       1 day ago
1     1 month ago
2      1 week ago
3       1 day ago
4    17 hours ago
Name: Post_time, dtype: object

* As shown in the above we have values like jobs posted 1 day ago, 1 month ago, 17 hours ago and so on.
* In order to convert it into datetime format, I have created following function::
    * if it has days in it then it will just returns (number of days)
    * if it has weeks in it then it will just returns (number of days) * 7
    * if it has months in it then it will just returns (number of days) * 30
    * if it has hours in it then it will just returns (number of days) / 30
    * if it has minutes in it then it will just returns (number of days) / (24 * 60)
    * if it has years in it then it will just returns (number of days) / 365

Once we have done all the calculations, we will get the values in only days format and that can be converted to date format [YYYY-MM-DD]

In [6]:
def convert_post_time(post_time):
    num = int(re.findall(r'\d+', post_time)[0])  # Extract the numeric value from the string
    if 'day' in post_time:
        return num
    elif 'week' in post_time:
        return num * 7
    elif 'month' in post_time:
        return num * 30
    elif 'hour' in post_time:
        return num / 24
    elif 'minute' in post_time:
        return num / (24 * 60)
    elif 'year' in post_time:
        return num * 365
    else:
        return None

DS_jobs['Post_time'] = DS_jobs['Post_time'].apply(convert_post_time)
DS_jobs['Post_time'] = DS_jobs['Post_time'].astype(int)  # Converting to numeric

# Calculating the actual date by subtracting the number of days from the current date
DS_jobs['Days_posted'] = (datetime.now() - pd.to_timedelta(DS_jobs['Post_time'], unit='d')).dt.strftime('%Y-%m-%d')

# Converting data type into datetime format 
DS_jobs['Days_posted'] = pd.to_datetime(DS_jobs['Days_posted']) 


In [8]:
DS_jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2346 entries, 0 to 2345
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Job_title         2346 non-null   object        
 1   Company           2346 non-null   object        
 2   Job_location      2346 non-null   object        
 3   Post_time         2346 non-null   int32         
 4   Applicants_count  2346 non-null   int64         
 5   Job_description   2346 non-null   object        
 6   Seniority_level   2346 non-null   object        
 7   Employment_type   2346 non-null   object        
 8   Job_function      2346 non-null   object        
 9   Industries        2346 non-null   object        
 10  Days_posted       2346 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int32(1), int64(1), object(8)
memory usage: 192.6+ KB


* From the Datys_Posted datetime variable we are extacting the Months, days and Quarter for EDA.

In [9]:
DS_jobs['Days_posted_Months'] = DS_jobs['Days_posted'].dt.month # Extracting Months
DS_jobs['Days_posted_Days'] = DS_jobs['Days_posted'].dt.day  # Extracting Days
DS_jobs['Days_posted_Quarter'] = DS_jobs['Days_posted'].dt.quarter # Extracting Quarter

**Demographics**

* From the Job_location feature we are segregating the Cities and state codes for better visualization

In [11]:

# Extracting city name
DS_jobs['City'] = DS_jobs['Job_location'].str.extract(r'([a-zA-Z\s]+),')

# Extracting state code
DS_jobs['State'] = DS_jobs['Job_location'].str.extract(r'(\b[A-Z]{2}\b)')

# Displaying the updated DataFrame
DS_jobs.head()

,Job_title,Company,Job_location,Post_time,Applicants_count,Job_description,Seniority_level,Employment_type,Job_function,Industries,Days_posted,Days_posted_Months,Days_posted_Days,Days_posted_Quarter,City,State
0,Machine Learning Engineer,Patterned Learning AI,"San Francisco, CA",1,58,", Full-Time, Salary $60K-$70K\n\n\n\n\nMinimum...",Entry level,Full-time,Engineering and Information Technology,IT Services and IT Consulting,2023-07-10,7,10,3,San Francisco,CA
1,Machine Learning Engineer,Neural Magic,"Somerville, MA",30,131,Neural Magic is an early-stage AI software com...,Associate,Full-time,Engineering and Information Technology,"Technology, Information and Internet",2023-06-11,6,11,2,Somerville,MA
2,Data Scientist,Patterned Learning AI,"New York, United States",7,250,This is a remote position.\n\n\n\n\nEntry-Leve...,Entry level,Full-time,Engineering and Information Technology,IT Services and IT Consulting,2023-07-04,7,4,3,New York,NaN
3,Data Scientist,Patterned Learning AI,"San Francisco, CA",1,49,"Junior Data Scientist Engineer, Full-Time, Sal...",Entry level,Full-time,Engineering and Information Technology,IT Services and IT Consulting,2023-07-10,7,10,3,San Francisco,CA
4,Data Scientist,Patterned Learning AI,"New York, NY",0,56,"Entry-Level Data Scientist Engineer - US, Full...",Entry level,Full-time,Engineering and Information Technology,IT Services and IT Consulting,2023-07-11,7,11,3,New York,NY
